# Infopillar Solution Pvt Ltd
# Task 1 : Fake News Detection Project       
# Author : Shubhangi Nannware
# Dataset:  https://bit.ly/3FxCSC4

# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shubhangi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# printing English Stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# loading dataset from local directory

In [4]:
dataset = pd.read_csv('C:/Users/shubhangi/Desktop/news.csv')

# printing first 4 rows of dataset

In [5]:
dataset.head(4)

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE


## Checking number of rows and columns of dataset

In [6]:
dataset.shape

(6335, 4)

# Labelling 'Real' News As '0' & 'Fake' News As '1'

In [7]:
label = {"REAL": 0, "FAKE": 1}
dataset['label'] = dataset['label'].map(label)
dataset.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",1
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,1
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,0
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",1
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,0


## checking number of null values from dataset

In [8]:
dataset.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

# Using Porter Stemming algorithm to remove commener morphological words and affixes of words with same meaning

In [9]:
stmg = PorterStemmer()

In [10]:
def stemming(title):
    stemmed_title = re.sub('[^a-zA-Z]',' ',title)
    stemmed_title = stemmed_title.lower()
    stemmed_title = stemmed_title.split()
    stemmed_title = [stmg.stem(word) for word in stemmed_title if not word in stopwords.words('english')]
    stemmed_title = ' '.join(stemmed_title)
    return stemmed_title

In [11]:
dataset['title'] = dataset['title'].apply(stemming)

In [12]:
print(dataset['title'])

0                                      smell hillari fear
1       watch exact moment paul ryan commit polit suic...
2                           kerri go pari gestur sympathi
3          berni support twitter erupt anger dnc tri warn
4                           battl new york primari matter
                              ...                        
6330       state depart say find email clinton specialist
6331                        p pb stand plutocrat pentagon
6332             anti trump protest tool oligarchi inform
6333    ethiopia obama seek progress peac secur east a...
6334                jeb bush suddenli attack trump matter
Name: title, Length: 6335, dtype: object


# Removing column label from dataset

In [13]:
X = dataset.drop(columns='label', axis=1)
Y = dataset['label']

In [14]:
print(X)
print(Y)

      Unnamed: 0                                              title  \
0           8476                                 smell hillari fear   
1          10294  watch exact moment paul ryan commit polit suic...   
2           3608                      kerri go pari gestur sympathi   
3          10142     berni support twitter erupt anger dnc tri warn   
4            875                      battl new york primari matter   
...          ...                                                ...   
6330        4490     state depart say find email clinton specialist   
6331        8062                      p pb stand plutocrat pentagon   
6332        8622           anti trump protest tool oligarchi inform   
6333        4021  ethiopia obama seek progress peac secur east a...   
6334        4330              jeb bush suddenli attack trump matter   

                                                   text  
0     Daniel Greenfield, a Shillman Journalism Fello...  
1     Google Pinterest Digg Lin

In [15]:
X = dataset['title'].values
Y = dataset['label'].values

In [16]:
print(X)

['smell hillari fear'
 'watch exact moment paul ryan commit polit suicid trump ralli video'
 'kerri go pari gestur sympathi' ...
 'anti trump protest tool oligarchi inform'
 'ethiopia obama seek progress peac secur east africa'
 'jeb bush suddenli attack trump matter']


In [17]:
print(Y)

[1 1 0 ... 1 0 0]


In [18]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [19]:
print(X)

  (0, 5631)	0.7649346276717717
  (0, 2817)	0.31703295849951846
  (0, 2230)	0.5606827254461214
  (1, 6671)	0.27855534550881683
  (1, 6571)	0.21736822482437437
  (1, 6315)	0.13561903129801023
  (1, 5933)	0.3501812302002013
  (1, 5263)	0.29163264316148546
  (1, 4905)	0.29382045650093463
  (1, 4630)	0.24403179029804972
  (1, 4462)	0.26757273402295384
  (1, 3939)	0.3279355637571237
  (1, 2101)	0.44595444447238664
  (1, 1212)	0.3538286538491304
  (2, 6015)	0.5345737509664292
  (2, 4428)	0.37615749278439464
  (2, 3335)	0.41740393299177314
  (2, 2576)	0.3357697230253524
  (2, 2536)	0.5345737509664292
  (3, 6658)	0.3169347988352881
  (3, 6359)	0.3719244073533966
  (3, 6288)	0.31487349957846533
  (3, 5956)	0.2780361491213491
  (3, 2048)	0.44483942568507917
  (3, 1764)	0.35074544989300727
  :	:
  (6330, 1133)	0.2086414599045263
  (6331, 5796)	0.38445288563338503
  (6331, 4611)	0.5530926787707366
  (6331, 4493)	0.45939150101458504
  (6331, 4474)	0.5790025183139886
  (6332, 6315)	0.177445191694796


# Splitting data for training and testing

In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.15, stratify=Y, random_state=2)

In [21]:
model = LogisticRegression()

In [22]:
model.fit(X_train, Y_train)

LogisticRegression()

# accuracy score on the training data

In [23]:

X_train_predict = model.predict(X_train)
training_accuracy = accuracy_score(X_train_predict, Y_train)

In [24]:
print('Accuracy score of the training data : ', training_accuracy)

Accuracy score of the training data :  0.9112184249628529


In [25]:
# accuracy score on the test data
X_test_predict = model.predict(X_test)
test_accuracy = accuracy_score(X_test_predict, Y_test)

In [26]:
print('Accuracy score of the test data : ', test_accuracy)

Accuracy score of the test data :  0.8023133543638276


In [27]:
X_new = X_test[2]

pred = model.predict(X_new)
print(pred)

if (pred[0]==0):
    print('The news is Real')
else:
    print('The news is Fake')

[0]
The news is Real


In [28]:
print(Y_test[3])

1


In [29]:
#The accuracy score of Fake news detection model is 80.23%.